In [18]:
import os
import json
import pandas as pd
import traceback

In [19]:
from langchain.chat_models import ChatOpenAI

In [ ]:
#WE CAN 3 WAYS TO IMPORT openaiapi
#1. we can use os.getenv(yourkey)
#2. export yourkey in terminal
#3. create .env file in your roo folder acces from it

In [20]:
#for loading the api key which was presnet in .env
from dotenv import load_dotenv


In [21]:
#tell will tell whether any key is preesent or not 
load_dotenv()

True

In [22]:
key = os.getenv("OPEN_API_KEY")

In [ ]:
print(key)

In [23]:
#thisi is my llm
#get your openai key in openaiplaygrounf.com
llm = ChatOpenAI(openai_api_key=key,model_name="gpt-3.5-turbo",temperature=0.5)

In [25]:
from langchain.llms import OpenAI        #
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback   #it is very important
import PyPDF2 

In [ ]:
#whatever subject im giving to my model it can able to produce questions/texts

In [29]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}


In [30]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [31]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )


In [32]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)  #here we create chain to my llm and prompt

In [33]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [34]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [35]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [37]:
#now creating a sequence chain between quiz chain and review_chain

generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [ ]:
#now we have to provide data 

In [38]:
file_path = r"C:\Users\poppo\Desktop\github ai\end to end by vivek\repo-20-generative-ai-demo-by-me\6-project-mcqgen\data.txt"

In [39]:
file_path

'C:\\Users\\poppo\\Desktop\\github ai\\end to end by vivek\\repo-20-generative-ai-demo-by-me\\6-project-mcqgen\\data.txt'

In [40]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [41]:
#here is my entire data
TEXT

'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[13] Other researchers who have studied human cognitive systems contrib

In [42]:
#above repsonse joson file in dictonary

#serializa the python dictiniary into a JSON-faormatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [44]:
NUMBER=5 
SUBJECT="machine learning"
TONE="simple"


In [45]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:          #if we want to track input token and output token each and every thing we will get using the get_openai_call back u can use above link for reference
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used 

In [46]:
#these are the benifts below we can track progress of apis of trained lln using get openai callback 
print(f"Total Tokens:{cb.total_tokens}")  
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:2505
Prompt Tokens:1794
Completion Tokens:711
Total Cost:0.004113


In [47]:
response

{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[13] Other researchers who have studied human cognitive system

In [72]:
quiz = response.get("quiz")

In [74]:
quiz = json.loads(quiz)

In [75]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [76]:
quiz_table_data

[{'MCQ': 'Who coined the term machine learning?',
  'Choices': 'a: Arthur Samuel | b: Donald Hebb | c: Walter Pitts | d: Warren McCulloch',
  'Correct': 'Arthur Samuel'},
 {'MCQ': 'What was the earliest machine learning model introduced by Arthur Samuel?',
  'Choices': 'a: Checkers program | b: Cybertron | c: Neural network | d: Pattern classification',
  'Correct': 'Checkers program'},
 {'MCQ': 'Which book introduced a theoretical neural structure for machine learning?',
  'Choices': 'a: The Organization of Behavior | b: Learning Machines | c: Computing Machinery and Intelligence | d: Learning from Data',
  'Correct': 'The Organization of Behavior'},
 {'MCQ': "In which decade was an experimental 'learning machine' called Cybertron developed?",
  'Choices': 'a: 1950s | b: 1960s | c: 1970s | d: 1980s',
  'Correct': '1960s'},
 {'MCQ': 'What are the two objectives of modern-day machine learning?',
  'Choices': 'a: Data classification and pattern recognition | b: Data analysis and decision

In [78]:
quiz = pd.DataFrame(quiz_table_data)

In [79]:
quiz.to_csv("machinelearning.csv",index = False)

In [80]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'02_03_2024_16_29_39'